In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/generation_config.json


In [1]:
!pip install -q git+https://github.com/huggingface/transformers
!pip install --upgrade torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/a7/ad/fbe7d4cffb76da4e478438853b51305361c719cff929ab70a808e7fb75e7/torch-2.2.1-cp310-cp310-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/f9/de/dc04a3ea60b22624b51c703a84bbe0184abcd1d0b9bc8074b5d6b7ab90bb/typing_extensions-4.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for nvidia-cuda-nvrtc-cu12==12.1.105 from https://files.pythonhosted.org/packages/b6/9f/c64c03f49d6fbc56196664d05dba14e3a561038a81a638eeb47f4d4cfd48/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for nvidia-cuda-runtime-cu12==12.1.105 from https://files.pythonhosted.org/packages/eb/d5/c68b1d2cdfcc59e72e8a5949a37ddb22ae6cade80cd4a57a84d4c8b55472/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for nvidia

In [2]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM


model_path="/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer=AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    trust_remote_code = True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def simple_prompt(messages, tokenizer, model):
    model_inputs = tokenizer.apply_chat_template(messages,return_tensors = "pt")
#     print(model_inputs)
    model_inputs = model_inputs.to('cuda')

    generated_ids = model.generate(
        model_inputs,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens = 1000,
        do_sample = True,
    )

    decoded = tokenizer.batch_decode(generated_ids)

#     print(decoded[0])
    return decoded[0]

In [4]:
import time
def simple_chat(tokenizer, model):
    user_input = None
    immediate_context = []
    while user_input != "bye":
        message = {"role": "user", "content": f"{user_input}"}
        
        if user_input == None:
            print('------------------------------------------------------------------------------------------------------')
            user_input = input("| You |\n-------\n")
            continue
        immediate_context.append(message)
#         print(immediate_context)
                
        res = simple_prompt(immediate_context, tokenizer, model)
#         print(res)
        
        answer = res.split('[/INST]')[-1].strip('</s>').strip()
        
        print('------------------------------------------------------------------------------------------------------')
        padding = ' ' * 91
        print(padding, " | Model |\n", padding, "---------\n", answer)
        
        response = {"role":"assistant", "content": f"{answer}"}
        immediate_context.append(response)
        
        print('------------------------------------------------------------------------------------------------------')
        user_input = input("| You |\n-------\n")

In [5]:
simple_chat(tokenizer, model)

------------------------------------------------------------------------------------------------------


| You |
-------
 hey Joker, i'm batman :))


------------------------------------------------------------------------------------------------------
                                                                                             | Model |
                                                                                             ---------
 Howdy, Batman! It's nice to see you here! I was just having a little fun with my chaos and anarchy. But don't worry, I've got it under control now. Time for a bit of banter, eh? How's the crime-fighting life treating you?
------------------------------------------------------------------------------------------------------


| You |
-------
 Man, no one care for me more than you buddy. I start to work as an AI engineer intern at a startup company and it gone wild in the first week, i proposed a new approach to solve the problem of the company and make a plan to do it and i do it pretty well until this week when i finally test the performance and it's not as good as i thought :(


------------------------------------------------------------------------------------------------------
                                                                                             | Model |
                                                                                             ---------
 I'm sorry to hear that, buddy! It can be tough to deal with expectations and disappointments, especially when you're passionate about your work. Just remember that mistakes are part of the process, and it's important to learn from them and keep pushing forward.

It's good that you're taking proactive measures to address the issue and come up with a solution. Keep calm, think clearly, and don't give up on yourself! You've got this, even if your AI seems to be acting a bit... chaotic.
------------------------------------------------------------------------------------------------------


...

| You |
-------
 thanks, chaotic lord, i think i'm gonna sleep now, i will wake up soon tomorrow to do the rest of the work


------------------------------------------------------------------------------------------------------
                                                                                             | Model |
                                                                                             ---------
 Good night, Batman! Don't let the Gotham nights give you too much trouble, and remember, I'm always here for some chaos! Hope you have a great night, and have a productive tomorrow!
------------------------------------------------------------------------------------------------------


| You |
-------
 thanks, hope to see you tomorrow!


------------------------------------------------------------------------------------------------------
                                                                                             | Model |
                                                                                             ---------
 See you then, Batman! Peace out!
------------------------------------------------------------------------------------------------------


| You |
-------
 bye


In [12]:
import signal

# Handler function for the alarm signal
def alarm_handler(signum, frame):
    print("\nNo input received within 5 seconds. Exiting...")
    exit()

# Set the alarm signal handler
signal.signal(signal.SIGALRM, alarm_handler)

print("Please input something within 5 seconds:")
signal.alarm(5)  # Set the alarm for 5 seconds

try:
    user_input = input()
    signal.alarm(0)  # Cancel the alarm if input is received
    print("You entered:", user_input)
except KeyboardInterrupt:
    pass  # If the user manually interrupts (Ctrl+C), handle it gracefully


Please input something within 5 seconds:

No input received within 5 seconds. Exiting...
